In [15]:
import pyomo.environ as pymo
import numpy as np

In [23]:
n_regions = 5
pi = [1/n_regions for i in range(n_regions)]

model = None
model = pymo.ConcreteModel()

#some parameters
#model.n = pymo.Param(mutable=True)
model.n = n_regions

#model.pi = pymo.Param([i for i in range(n_regions)])#, mutuable=True)
model.pi = pi

XMIN = 0; XMAX=50; YMIN=0; YMAX=50
#add x and y variables
model.x = pymo.Var([i for i in range(n_regions)], bounds=(XMIN, XMAX))
model.y = pymo.Var([i for i in range(n_regions)], bounds=(YMIN, YMAX))

#model.s = pymo.Var([i for i in range(int(n_regions*(n_regions+1)/2))])

# def AvgSwtichingTime(model):
#     terms = [ model.pi[i]*model.pi[j]*model.s[i] for i in range(model.n) for j in range(i+1, model.n)]
#     return sum(terms)

def AvgSwtichingTime(model):
    terms = [ model.pi[i]*model.pi[j]*((model.x[i]-model.x[j])**2 + (model.y[i]-model.y[j])**2)**0.5 for i in range(model.n) for j in range(i+1, model.n)]
    return sum(terms)

model.objective = pymo.Objective(rule=AvgSwtichingTime)

#constrain (x,y) to be in relay regions
mks = [mk for reg in self.cregions]
eta_idxs = np.cumsum(mks)
model.eta = pymo.Var([i in range(eta_idxs[-1])], domain=pymo.Binary)

def regional_constraint(model, start, end):
    return (sum(model.eta[start:end]) >= 1)

model.InRegions=pymo.ConstraintList()
#at least one must be set
start = 0
end = 0
for i in range(n_regions):
    start = end
    end = eta_idxs[i]
    model.InRegions.add(model.eta[start:end] >= 1)
    
#and finally, our hefty linear constraints
    
def PolyBoundary(model, Aik, bik, i, j, l):
    return (Aik[j,0]*model.x[i] + Aik[j,1]*model.y[i] +C*model.eta[l] <= bik[j,0])
    
C = 10000 #actually need to find a value of this constant
l = 0
for i in range(n_regions):
    reg = self.cregions[i]
    for poly in reg.polygons:
        for cnvx in poly.cnvx_partition:
            Aik, bik = cnvx.to_linear_constraints()
            bik[:,0] += C
            
            for j in range(len(bik)):
                model.Constraint(rule=lambda model: PolyBoundary(model, Aik, bik, i, j, l))
            l+=1

In [5]:
model